### Gauss消元法
高斯消元法是一种用于求解线性方程组的方法，通过行变换将线性方程组转化为上三角形式，然后回代求解。

In [1]:
def gaussian_elimination(matrix):
    n = len(matrix)

    for i in range(n):
        # 找到主元素的行
        max_row = i
        for j in range(i + 1, n):
            if abs(matrix[j][i]) > abs(matrix[max_row][i]):
                max_row = j

        # 交换行
        matrix[i], matrix[max_row] = matrix[max_row], matrix[i]

        # 主元素归一化
        pivot = matrix[i][i]
        for j in range(i, n + 1):
            matrix[i][j] /= pivot

        # 消元
        for j in range(i + 1, n):
            factor = matrix[j][i]
            for k in range(i, n + 1):
                matrix[j][k] -= factor * matrix[i][k]

    # 回代求解
    solution = [0] * n
    for i in range(n - 1, -1, -1):
        solution[i] = matrix[i][n]
        for j in range(i + 1, n):
            solution[i] -= matrix[i][j] * solution[j]

    return solution

# 示例
matrix = [
    [2, -3, 5, -1, 3],
    [1, 4, 2, -3, 7],
    [-2, 4, -3, -7, -1],
    [8, 0, -2, 1, 8]
]

solution = gaussian_elimination(matrix)
print("Solution:", solution)

Solution: [1.191304347826087, 1.1157190635451504, 0.811371237458194, 0.09230769230769226]


### [雅可比迭代法（Jacobi Iteration）](https://zh.wikipedia.org/wiki/%E9%9B%85%E5%8F%AF%E6%AF%94%E6%B3%95)

是一种用于解线性方程组的迭代方法，适用于对角占优矩阵。该方法通过不断迭代来逐步逼近线性方程组的解。

假设线性方程组为 Ax = b，其中 A 是系数矩阵，x 是未知向量，b 是常数向量。雅可比迭代法的基本思想是将系数矩阵 A 拆分为一个对角矩阵 D 和一个非对角矩阵 R，然后不断迭代以下公式来逼近解 x：

x^(k+1)^ = D^(-1)^ * (b - R * x^(k)^)

其中 x^(k)^ 是第 k 次迭代的近似解。

In [3]:
import numpy as np

def jacobi_iteration(A, b, max_iterations=100, tolerance=1e-6):
    n = len(A)
    x = np.zeros(n)

    for _ in range(max_iterations):
        x_new = np.zeros(n)
        for i in range(n):
            sum_ax = np.dot(A[i, :], x) - A[i, i] * x[i]
            x_new[i] = (b[i] - sum_ax) / A[i, i]

        if np.linalg.norm(x_new - x) < tolerance:
            return x_new

        x = x_new

    return x

# 示例
A = np.array([
    [2, 1],
    [5, 7]
])

b = np.array([11, 13])

solution = jacobi_iteration(A, b)
print("Solution:", solution)


ModuleNotFoundError: No module named 'numpy'

### [拉格朗日插值函数算法](https://zh.wikipedia.org/zh-cn/%E6%8B%89%E6%A0%BC%E6%9C%97%E6%97%A5%E6%8F%92%E5%80%BC%E6%B3%95)

拉格朗日插值法是一种用于在给定数据点的情况下，通过一个多项式来逼近这些数据点的方法。它可以用来估计两个数据点之间的值，或者用来插值一组数据点来构造一个多项式函数。

假设有一组已知的数据点 (x_i, y_i)，其中 x_i 是自变量，y_i 是因变量。拉格朗日插值法的目标是找到一个多项式 P(x) = a_0 + a_1*x + a_2*x^2^ + ... + a_n*x^n^，使得 P(x_i) = y_i 对所有数据点成立。

多项式 P(x) 的表达式为：

P(x) = Σ (y_i * L_i(x))

其中 L_i(x) 是拉格朗日基函数：

L_i(x) = Π [(x - x_j) / (x_i - x_j)]，j ≠ i

In [6]:
def lagrange_interpolation(x_vals, y_vals, x):
    n = len(x_vals)
    result = 0

    for i in range(n):
        term = y_vals[i]
        for j in range(n):
            if j != i:
                term *= (x - x_vals[j]) / (x_vals[i] - x_vals[j])
        result += term

    return result

# 示例
x_vals = [4, 5, 6]
y_vals = [10, 5.25, 1]

x = 18
y = lagrange_interpolation(x_vals, y_vals, x)
print(f"Interpolated value at x = {x}: {y}")

Interpolated value at x = 18: -11.0


## 图论相关算法

### 最小生成树

In [16]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if x != self.parent[x]:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        root_x, root_y = self.find(x), self.find(y)
        if root_x == root_y:
            return False

        if self.rank[root_x] < self.rank[root_y]:
            self.parent[root_x] = root_y
        elif self.rank[root_x] > self.rank[root_y]:
            self.parent[root_y] = root_x
        else:
            self.parent[root_x] = root_y
            self.rank[root_y] += 1

        return True

def kruskal_mst(graph):
    n = len(graph)
    edges = []
    for u in range(n):
        for v, weight in graph[u]:
            edges.append((weight, u, v))
    edges.sort()

    mst = []
    dsu = DisjointSet(n)
    for weight, u, v in edges:
        if dsu.union(u, v):
            mst.append((u, v, weight))

    return mst

# Example usage
n = 4  # Number of vertices
graph = [
    [(1, 5), (3, 10)],  # (neighbor, weight)
    [(0, 5), (2, 3)],
    [(1, 3), (3, 7)],
    [(0, 10), (2, 7)]
]

mst = kruskal_mst(graph)
print("Minimum Spanning Tree Edges:")
for u, v, weight in mst:
    print(f"{u} -- {v} : {weight}")


Minimum Spanning Tree Edges:
1 -- 2 : 3
0 -- 1 : 5
2 -- 3 : 7


In [15]:
# Prim算法
import heapq

def prim_mst(graph):
    n = len(graph)
    mst = []
    visited = set()
    start_vertex = 0  # 选择一个起始顶点

    # 使用优先队列来存储顶点和权重的信息
    priority_queue = [(0, start_vertex)]  # (权重, 顶点)

    while priority_queue:
        weight, vertex = heapq.heappop(priority_queue)
        if vertex not in visited:
            visited.add(vertex)
            if vertex != start_vertex:
                mst.append((vertex, parent[vertex], weight))
            for neighbor, edge_weight in graph[vertex]:
                if neighbor not in visited:
                    heapq.heappush(priority_queue, (edge_weight, neighbor))
                    parent[neighbor] = vertex

    return mst

# Example usage
n = 4  # Number of vertices
graph = [
    [(1, 5), (3, 10)],  # (neighbor, weight)
    [(0, 5), (2, 3)],
    [(1, 3), (3, 7)],
    [(0, 10), (2, 7)]
]

parent = [None] * n  # 存储每个顶点的父节点，用于构建最小生成树

mst = prim_mst(graph)
print("Minimum Spanning Tree Edges:")
for u, v, weight in mst:
    print(f"{u} -- {v} : {weight}")


Minimum Spanning Tree Edges:
1 -- 0 : 5
2 -- 1 : 3
3 -- 2 : 7


### 最短路的Dijkstra算法

In [17]:
import heapq

def dijkstra(graph, start):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0
    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

# Example usage
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}

start_vertex = 'A'
shortest_distances = dijkstra(graph, start_vertex)
print("Shortest distances from vertex", start_vertex)
for vertex, distance in shortest_distances.items():
    print(f"To vertex {vertex}: {distance}")

Shortest distances from vertex A
To vertex A: 0
To vertex B: 1
To vertex C: 3
To vertex D: 4


In [18]:
import sys
max = sys.maxsize

vertices_number = 6
adjacency_matrix = [
    [0, 1, 10, -1, -1, 2],
    [10, 0, 1, -1, -1, -1],
    [1, 10, 0, -1, -1, -1],
    [-1, -1, 2, 0, 1, 10],
    [-1, -1, -1, 10, 0, 1],
    [-1, -1, -1, 1, 10, 0]]
start = []
dest = ["2", "5"]
key = []


def init_keys(s: int):
    global key
    key = [ max ] * vertices_number
    key[s] = 0


def dijkstra(from_vertex, dest_vertex):
    fid = int(from_vertex) - 1
    tid = int(dest_vertex) - 1
    init_keys(fid)
    rel = [fid]
    min_vertex = fid
    hop_path = {}

    while len(rel) <= vertices_number and min_vertex != tid:
        for i in range(vertices_number):
            if i != min_vertex and i not in rel and \
                adjacency_matrix[min_vertex][i] > 0 \
                and key[i] > key[min_vertex] + adjacency_matrix[min_vertex][i]:
                key[i] = key[min_vertex] + adjacency_matrix[min_vertex][i]
                hop_path.update({i + 1: {"from": min_vertex + 1, "cost": adjacency_matrix[min_vertex][i]}})

        if min_vertex not in rel:
            rel.append(min_vertex)

        min_vertex = tid
        for i in range(vertices_number):
            if i not in rel and key[i] < key[min_vertex]:
                min_vertex = i

    if len(hop_path) == 0 or int(dest_vertex) not in hop_path:
        return -1, -1
    else:
        next_hop = int(dest_vertex)
        path_str = dest_vertex
        while hop_path[next_hop]["from"] != int(from_vertex):
            cost = hop_path[next_hop]["cost"]
            next_hop = hop_path[next_hop]["from"]
            path_str =  "{} -({})-> {}".format(str(next_hop), cost ,path_str)
        path_str =  "{} -({})-> {}".format(str(hop_path[next_hop]["from"]), hop_path[next_hop]["cost"], path_str)

        return key[tid], path_str



def find_shortest_router():
    for s in start:
        print("Forwarding Table for {}".format(s))
        print("{:>10} {:>10}       {}".format("To", "Cost", "Path"))
        for d in dest:
            c, n = dijkstra(s, d)
            print("{:>10} {:>10}       {}".format(d, c, n))


def main():
    for i in range(1, vertices_number + 1):
        if str(i) not in dest:
            start.append(str(i))
    find_shortest_router()

if __name__ == '__main__':
    main()

Forwarding Table for 1
        To       Cost       Path
         2          1       1 -(1)-> 2
         5          4       1 -(2)-> 6 -(1)-> 4 -(1)-> 5
Forwarding Table for 3
        To       Cost       Path
         2          2       3 -(1)-> 1 -(1)-> 2
         5          5       3 -(1)-> 1 -(2)-> 6 -(1)-> 4 -(1)-> 5
Forwarding Table for 4
        To       Cost       Path
         2          4       4 -(2)-> 3 -(1)-> 1 -(1)-> 2
         5          1       4 -(1)-> 5
Forwarding Table for 6
        To       Cost       Path
         2          5       6 -(1)-> 4 -(2)-> 3 -(1)-> 1 -(1)-> 2
         5          2       6 -(1)-> 4 -(1)-> 5


### Ford最短路算法

In [19]:
def bellman_ford(graph, start):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0

    for _ in range(len(graph) - 1):
        for vertex in graph:
            for neighbor, weight in graph[vertex].items():
                if distances[vertex] + weight < distances[neighbor]:
                    distances[neighbor] = distances[vertex] + weight

    # Check for negative-weight cycles
    for vertex in graph:
        for neighbor, weight in graph[vertex].items():
            if distances[vertex] + weight < distances[neighbor]:
                raise ValueError("Graph contains negative-weight cycle")

    return distances

# Example usage
graph = {
    'A': {'B': -1, 'C': 4},
    'B': {'C': 3, 'D': 2, 'E': 2},
    'C': {},
    'D': {'B': 1, 'C': 5},
    'E': {'D': -3}
}

start_vertex = 'A'
shortest_distances = bellman_ford(graph, start_vertex)
print("Shortest distances from vertex", start_vertex)
for vertex, distance in shortest_distances.items():
    print(f"To vertex {vertex}: {distance}")


Shortest distances from vertex A
To vertex A: 0
To vertex B: -1
To vertex C: 2
To vertex D: -2
To vertex E: 1
